# Push Model from Pre-Prod to Prod Deployment Space
This notebook pushes a model from a pre-prod space to a prod space as part of a CICD flow.

### Load environment variables:

In [ ]:
import os
pre_prod_space_name = os.environ.get('PRE_PROD_SPACE_NAME').replace("\"","")
pre_prod_deployment_name = os.environ.get('PRE_PROD_DEPLOYMENT_NAME').replace("\"","")
prod_space_name = os.environ.get('PROD_SPACE_NAME').replace("\"","")
prod_deployment_name = os.environ.get('PROD_DEPLOYMENT_NAME').replace("\"","")

---------------------------------------------------------------------------------------------------------------------

### Authenticate to WML Client

In [2]:
from ibm_watson_machine_learning import APIClient
import os

In [3]:
token = os.environ['USER_ACCESS_TOKEN']
    
WML_CREDENTIALS = {
        "token": token,
        "instance_id": "wml_local",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": '3.5'
    }
wml_client = APIClient(WML_CREDENTIALS)

## Determine Pre-Prod Space ID and Model ID

In [4]:
pre_prod_space = [space['metadata']['id'] for space in wml_client.spaces.get_details()['resources'] if space['entity']['name'] == pre_prod_space_name]
space_id = pre_prod_space[0]
prod_space = [space['metadata']['id'] for space in wml_client.spaces.get_details()['resources'] if space['entity']['name'] == prod_space_name]
prod_space_id = prod_space[0]
wml_client.set.default_space(space_id)

'SUCCESS'

In [5]:
deps=wml_client.deployments.get_details()

In [6]:
pre_prod_model_id = [x['entity']['asset']['id'] for x in deps['resources'] if x['metadata']['name']==pre_prod_deployment_name][0]

In [7]:
os.getcwd()

'/home/wsuser/work'

## Download Model Binary From Space in Memory 

In [22]:
model_name = f'{pre_prod_deployment_name.lower().replace(" ","_")}.tar.gz'

In [23]:
model_name

'flight_delay_pre_prod_deployment.tar.gz'

In [24]:
model_filepath = os.getcwd() + '/' + model_name

In [17]:
model_filepath

'/home/wsuser/work/flight_delay_pre_prod_deployment.tar.gz'

In [28]:
if os.path.exists(model_filepath) == False:
    wml_client.repository.download(pre_prod_model_id,model_name)

### Get Pre-Prod Model Metadata

In [29]:
model_metadata = wml_client.repository.get_details(pre_prod_model_id)

In [30]:
model_metadata

{'entity': {'content_import_state': 'completed',
  'hybrid_pipeline_software_specs': [{'id': '632d4b22-10aa-5180-88f0-f52dfb6444d7',
    'name': 'autoai-kb_3.1-py3.7'}],
  'pipeline': {'id': '717289ca-1b80-470a-b4e1-a42240762f4e'},
  'schemas': {'input': [{'fields': [{'name': 'DAY', 'type': 'integer'},
      {'name': 'DAY_OF_WEEK', 'type': 'integer'},
      {'name': 'ORIGIN_AIRPORT', 'type': 'other'},
      {'name': 'DESTINATION_AIRPORT', 'type': 'other'},
      {'name': 'DEPARTURE_DELAY', 'type': 'double'},
      {'name': 'TAXI_OUT', 'type': 'double'},
      {'name': 'DISTANCE', 'type': 'integer'}],
     'id': 'auto_ai_kb_input_schema'}],
   'output': []},
  'software_spec': {'id': '8c1a58c6-62b5-4dc4-987a-df751c2756b6',
   'name': 'hybrid_0.1'},
  'training_data_references': [{'connection': {},
    'id': 'train_flights_jan_2015.csv_shaped',
    'location': {'href': '/v2/assets/3f78da6c-04dd-499a-b8ce-dd60ccc9fa7f?project_id=6ece413e-4d52-4478-8ff1-f13382e836bb'},
    'type': 'data_as

## Create Model Asset in Prod Space 

In [31]:
target_space_id = prod_space_id
name = model_metadata['metadata']['name']
model_type = model_metadata['entity']['type']
sw_spec_name = model_metadata['entity']['software_spec']['name']

In [32]:
# Change default space 
wml_client.set.default_space(target_space_id)

'SUCCESS'

In [33]:
sw_spec_id = wml_client.software_specifications.get_id_by_name(sw_spec_name)

In [34]:
metadata = {
       wml_client.repository.ModelMetaNames.NAME: name,
       wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_id,
       wml_client.repository.ModelMetaNames.TYPE: model_type
}

In [35]:
stored_model_details = wml_client.repository.store_model(model_filepath, meta_props=metadata, training_data=None)

In [36]:
stored_model_details

{'entity': {'software_spec': {'id': '8c1a58c6-62b5-4dc4-987a-df751c2756b6',
   'name': 'hybrid_0.1'},
  'type': 'wml-hybrid_0.1'},
 'metadata': {'created_at': '2021-09-10T14:50:31.362Z',
  'id': '77c530ed-b5a8-4029-bc9c-c2105a94bfd9',
  'modified_at': '2021-09-10T14:50:33.154Z',
  'name': 'Airline_AutoAI_exp - P3 GradientBoostingClassifierEstimator',
  'owner': '1000330999',
  'space_id': '4676041f-95ab-46fc-8e94-8ed26498dd58'},
 'system': {'warnings': []}}

## Deploy Model in Prod Space

In [37]:
metadata = {
        wml_client.deployments.ConfigurationMetaNames.NAME: prod_deployment_name,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
model_id = stored_model_details["metadata"]["id"]
deployment_details = wml_client.deployments.create(artifact_uid=model_id, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '77c530ed-b5a8-4029-bc9c-c2105a94bfd9' started

#######################################################################################


initializing...........................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6e32875f-bc17-4121-bea9-5e38d6ed7e93'
------------------------------------------------------------------------------------------------


